# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 61 new papers today
          23 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/22 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.05792


extracting tarball to tmp_2211.05792...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.05799


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.05792/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tab6.tex' from 'tmp_2211.05792/tab6.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tab5.tex' from 'tmp_2211.05792/tab5.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py

...

Retrieving document from  https://arxiv.org/e-print/2211.06354


extracting tarball to tmp_2211.06354...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.06367


extracting tarball to tmp_2211.06367...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06194-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06194) | **15000 Ellipsoidal Binary Candidates in TESS: Orbital Periods, Binary  Fraction, and Tertiary Companions**  |
|| <mark>Matthew J. Green</mark>, et al. -- incl., <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *27 pages, 21 figures, submitted to MNRAS. The associated sample data will be released alongside the accepted paper*|
|**Abstract**| We present a homogeneously-selected sample of 15779 candidate binary systems with main sequence primary stars and orbital periods shorter than 5 days. The targets were selected from TESS full-frame image lightcurves on the basis of their tidally-induced ellipsoidal modulation. Spectroscopic follow-up suggests a sample purity of $83 \pm 13$ per cent. Injection-recovery tests allow us to estimate our overall completeness as $28 \pm 3$ per cent with $P_\mathrm{orb} < 3$ days and to quantify our selection effects. We estimate that $39 \pm 4$ per cent of our sample are contact binary systems, and we disentangle the period distributions of the contact and detached binaries. We derive the orbital period distribution of the main sequence binary population at short orbital periods, finding a distribution continuous with the log-normal distribution previously found for solar-type stars at longer periods, but with a significant steepening at $P_\mathrm{orb} \lesssim 3$ days, and a pile-up of contact binaries at $P_\mathrm{orb} \approx 0.4$ days. Companions in the period range 1--5 days are found to be an order of magnitude more frequent around stars hotter than $\approx 6250 K$ (the Kraft break) when compared to cooler stars, suggesting that magnetic braking plays an important role in shaping the temperature and period distributions. We detect resolved tertiary companions to $9.0 \pm 0.2$ per cent of our binaries with a median separation of 3200 AU. The frequency of tertiary companions rises to $29 \pm 5$ per cent among the systems with the shortest ellipsoidal periods. This large binary sample with quantified selection effects will be a powerful resource for future studies of detached and contact binary systems with $P_\mathrm{orb} < 5$ days. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06205-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06205) | **A sub-Neptune planet around TOI-1695 discovered and characterized with  SPIRou and TESS**  |
|| F. Kiefer, et al. -- incl., <mark>K. Hesse</mark>, <mark>H. Lewis</mark>, <mark>C.-F. Liu</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *15 pages (+9 appendix pages), 19 figures, 8 tables*|
|**Abstract**| TOI-1695 is a V-mag=13 M-dwarf star from the northern hemisphere at 45$\,$pc from the Sun, around which a 3.134-day periodic transit signal from a super-Earth candidate was identified in TESS photometry. With a transit depth of 1.3$\,$mmag, the radius of candidate TOI-1695.01 was estimated by the TESS pipeline to be 1.82$\,$R$_\oplus$ with an equilibrium temperature of $\sim$620$\,$K. We successfully detect a reflex motion of the star and establish it is due to a planetary companion at an orbital period consistent with the photometric transit period thanks to a year-long radial-velocity monitoring of TOI-1695 by the SPIRou infrared spectropolarimeter. We use and compare different methods to reduce and analyse those data. We report a 5.5-$\sigma$ detection of the planetary signal, giving a mass of $5.5$$\pm$$1.0\,$M$_\oplus$ and a radius of 2.03$\pm$0.18$\,$R$_\oplus$. We derive a mean equilibrium planet temperature of 590$\pm$90$\,$K. The mean density of this small planet of 3.6$\pm$1.1$\,$g$\,$cm$^{-3}$ is similar (1.7-$\sigma$ lower) than that of the Earth. It leads to a non-negligible fraction of volatiles in its atmosphere with $f_{H,He}$=0.28$^{+0.46}_{-0.23}$% or $f_\text{water}$=23$\pm$12%. TOI-1695$\,$b is a new sub-Neptune planet at the border of the M-dwarf radius valley that can help test formation scenarios for super-Earth/sub-Neptune-like planets. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05800-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05800) | **Stellar population analysis of MaNGA early-type galaxies: IMF dependence  and systematic effects**  |
|| M. Bernardi, et al. -- incl., <mark>J. R. Brownstein</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *22 pages, 23 figures, accepted for publication in MNRAS*|
|**Abstract**| We study systematics associated with estimating simple stellar population (SSP) parameters -- age, metallicity [M/H], $\alpha$-enhancement [$\alpha$/Fe] and IMF shape -- and associated $M_*/L$ gradients, of elliptical slow rotators (E-SRs), fast rotators (E-FRs) and S0s from stacked spectra of galaxies in the MaNGA survey. These systematics arise from (i) how one normalizes the spectra when stacking; (ii) having to subtract emission before estimating absorption line strengths; (iii) the decision to fit the whole spectrum or just a few absorption lines; (iv) SSP model differences (e.g. isochrones, enrichment, IMF). The MILES+Padova SSP models, fit to the H$_\beta$, $\langle$Fe$\rangle$, TiO$_{\rm 2SDSS}$ and [MgFe] Lick indices in the stacks, indicate that out to the half-light radius $R_e$: (a) ages are younger and [$\alpha$/Fe] values are lower in the central regions but the opposite is true of [M/H]; (b) the IMF is more bottom-heavy in the center, but is close to Kroupa beyond about $R_e/2$; (c) this makes $M_*/L$ about $2\times$ larger in the central regions than beyond $R_e/2$. While the models of Conroy et al. (2018) return similar [M/H] and [$\alpha$/Fe] profiles, the age and (hence) $M_*/L$ profiles can differ significantly even for solar abundances and a Kroupa IMF; different responses to non-solar abundances and IMF parametrization further compound these differences. There are clear (model independent) differences between E-SRs, E-FRs and S0s: younger ages and less enhanced [$\alpha$/Fe] values suggest that E-FRs and S0s are not SSPs, but relaxing this assumption is unlikely to change their inferred $M_*/L$ gradients significantly. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06354-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06354) | **Comparative Analysis of TRGBs (CATs) from Unsupervised, Multi-Halo-Field  Measurements: Contrast is Key**  |
|| J. Wu, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *Submitted to ApJ. Comments welcomed*|
|**Abstract**| The Tip of the Red Giant Branch (TRGB) is an apparent discontinuity in the color-magnitude diagram (CMD) along the giant branch due to the end of the red giant evolutionary phase and is used to measure distances in the local universe. In practice, the tip is often fuzzy and its localization via edge detection response (EDR) relies on several methods applied on a case-by-case basis. It is hard to evaluate how individual choices affect a distance estimation using only a single host field while also avoiding confirmation bias. To devise a standardized approach, we compare unsupervised, algorithmic analyses of the TRGB in multiple halo fields per galaxy, up to 11 fields for a single host and 50 fields across 10 galaxies, using high signal-to-noise stellar photometry obtained by the GHOSTS survey with the Hubble Space Telescope. We first optimize methods for the lowest field-to-field dispersion including spatial filtering to remove star forming regions, smoothing and weighting of the luminosity function, selection of the RGB by color, and tip selection based on the number of likely RGB stars and the ratio of stars above versus below the tip ($R$). We find $R$, which we call the tip `contrast', to be the most important indicator of the quality of EDR measurements; we find that field-to-field EDR repeatability varies from 0.3 mag to $\leq$ 0.05 mag for $R=4$ to 7, respectively, though less than half the fields reach the higher quality. Further, we find that $R$, which varies with the age/metallicity of the stellar population based on models, correlates with the magnitude of the tip (and after accounting for low internal extinction), i.e., a tip-contrast relation with slope of $-0.023\pm0.0046$ mag/ratio, a $\sim 5\sigma$ result that improves standardization of the TRGB. We discuss the value of consistent TRGB standardization across rungs for robust distance ladder measurements. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.06367-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.06367) | **The unaltered pulsar: GRO J1750-27, a super-critical X-ray neutron star  that does not blink an eye**  |
|| C. Malacaria, et al. -- incl., <mark>R. Rothschild</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *Accepted on A&A*|
|**Abstract**| When accreting X-ray pulsars (XRPs) undergo bright X-ray outbursts, their luminosity-dependent spectral and timing features can be analysed in detail. The XRP GRO J1750-27 recently underwent one of such episodes, during which it was observed with $NuSTAR$ and monitored with $NICER$. Such a data set is rarely available, as it samples the outburst over more than a month at a luminosity that is always exceeding ${\sim}5\times10^{37}\,$erg/s. This value is larger than the typical critical luminosity value, where a radiative shock is formed above the neutron star's surface. Our data analysis of the joint spectra returns a highly ($N_H\sim(5-8)\times10^{22}\,$cm$^{-2}$) absorbed spectrum showing a K$\alpha$ iron line, a soft blackbody component likely originating from the inner edge of the accretion disk, and confirms the discovery of one of the deepest cyclotron lines, at a centroid energy of ${\sim}44\,$keV corresponding to a magnetic field strength of $4.7\times10^{12}\,$G. This value is independently supported by the best-fit physical model for spectral formation in accreting XRPs which, in agreement with recent findings, favours a distance of $14$ kpc and also reflects a bulk-Comptonization dominated accretion flow. Contrary to theoretical expectations and observational evidence from other similar sources, the pulse profiles as observed by $NICER$ through the outburst raise, peak and decay remain remarkably steady. The $NICER$ spectrum, including the iron K$\alpha$ line best-fit parameters, also remain almost unchanged at all probed outburst stages, similar to the pulsed fraction behaviour. We argue that all these phenomena are linked and interpret them as resulting from a saturation effect of the accretion column's emission, which occurs in the high-luminosity regime. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05940-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05940) | **M Subdwarf Research III. Spectroscopic Diagnostics for Breaking  Parameter Degeneracy**  |
|| <mark>Shuo Zhang</mark>, et al. -- incl., <mark>Hua-Wei Zhang</mark>, <mark>Neda Hejazi</mark>, <mark>Yin-Bi Li</mark>, <mark>A-Li Luo</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *16 pages, 10 figures. Accepted to ApJ*|
|**Abstract**| To understand the parameter degeneracy of M subdwarf spectra at low resolution, we assemble a large number of spectral features in the wavelength range of 0.6-2.5 {\mu}m with band strength quantified by narrowband indices. Based on the index trends of BT-Settl model sequences, we illustrate how the main atmospheric parameters (Teff, log g, [M/H], and [alpha/Fe]) affect each spectral feature differently. Furthermore, we propose a four-step process to determine the four parameters sequentially, which extends the basic idea proposed by Jao et al. Each step contains several spectral features that break the degeneracy effect when determining a specific stellar parameter. Finally, the feasibility of each spectroscopic diagnostic with different spectral qualities is investigated. The result is resolution-independent down to R~200. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05972-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05972) | **Detection of Strongly Lensed Arcs in Galaxy Clusters with Transformers**  |
|| Peng Jia, et al. -- incl., <mark>Nan Li</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *Submitted to the Astronomical Journal, source code could be obtained from PaperData sponsored by China-VO group with DOI of 10.12149/101172. Cloud computing resources would be released under request*|
|**Abstract**| Strong lensing in galaxy clusters probes properties of dense cores of dark matter halos in mass, studies the distant universe at flux levels and spatial resolutions otherwise unavailable, and constrains cosmological models independently. The next-generation large scale sky imaging surveys are expected to discover thousands of cluster-scale strong lenses, which would lead to unprecedented opportunities for applying cluster-scale strong lenses to solve astrophysical and cosmological problems. However, the large dataset challenges astronomers to identify and extract strong lensing signals, particularly strongly lensed arcs, because of their complexity and variety. Hence, we propose a framework to detect cluster-scale strongly lensed arcs, which contains a transformer-based detection algorithm and an image simulation algorithm. We embed prior information of strongly lensed arcs at cluster-scale into the training data through simulation and then train the detection algorithm with simulated images. We use the trained transformer to detect strongly lensed arcs from simulated and real data. Results show that our approach could achieve 99.63 % accuracy rate, 90.32 % recall rate, 85.37 % precision rate and 0.23 % false positive rate in detection of strongly lensed arcs from simulated images and could detect almost all strongly lensed arcs in real observation images. Besides, with an interpretation method, we have shown that our method could identify important information embedded in simulated data. Next step, to test the reliability and usability of our approach, we will apply it to available observations (e.g., DESI Legacy Imaging Surveys) and simulated data of upcoming large-scale sky surveys, such as the Euclid and the CSST. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05897-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05897) | **The Star-Planet Activity Research CubeSat (SPARCS): Determining Inputs  to Planetary Habitability**  |
|| David R. Ardila, et al. -- incl., <mark>Joseph Llama</mark> |
|*Appeared on*| *2022-11-14*|
|*Comments*| *Presented at the 73rd International Astronautical Congress, 18-22 September 2022, Paris, France*|
|**Abstract**| Seventy-five billion low-mass stars in our galaxy host at least one small planet in their habitable zone (HZ). The stellar ultraviolet (UV) radiation received by the planets is strong and highly variable, and has consequences for atmospheric loss, composition, and habitability. SPARCS is a NASA-funded mission to characterize the quiescent and flare UV emission from low-mass stars, by observing 10 to 20 low-mass stars, over timescales of days, simultaneously in two UV bands: 153-171 nm and 260-300 nm. SPARCS Sun-synchronous terminator orbit allows for long periods of uninterrupted observations, reaching 10s of days for some targets. The payload consists of a 10 cm-class telescope, a dichroic element, UV detectors and associated electronics, a thermal control system, and an on-board processor. The payload is hosted on a Blue Canyon Technologies 6U CubeSat. SPARCS hosts several technology innovations that have broad applicability to other missions. The payload demonstrates the use of "2D-doped" (i.e., delta- and superlattice-doped) detectors and detector-integrated metal dielectric filters in space. This detector technology provides ~5x larger quantum efficiency than NASA's GALEX detectors. In addition, SPARCS' payload processor provides dynamic exposure control, automatically adjusting the exposure time to avoid flare saturation and to time-resolve the strongest stellar flares. A simple passive cooling system maintains the detector temperature under 238K to minimize dark current. The spacecraft bus provides pointing jitter smaller than 6", minimizing the impact of flat-field errors, dark current, and read-noise. All these elements enable competitive astrophysics science within a CubeSat platform. SPARCS is currently in the final design and fabrication phase (Phase C in the NASA context). It will be launched in 2024, for a primary science mission of one year. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.06194.md
    + _build/html/tmp_2211.06194/./figures/target_properties.png
    + _build/html/tmp_2211.06194/./figures/period_distribution_comparison.png
    + _build/html/tmp_2211.06194/./figures/contact_fraction_period.png
exported in  _build/html/2211.06205.md
    + _build/html/tmp_2211.06205/./toi_1695_gaia.png
    + _build/html/tmp_2211.06205/./TOI-1695_rvsvswin_DC_v6_periodogram.png
    + _build/html/tmp_2211.06205/./TOI-1695_fwhmvswin_DC_v6_periodogram.png
    + _build/html/tmp_2211.06205/./TOI-1695_bisvswin_DC_v6_periodogram.png
    + _build/html/tmp_2211.06205/./TIC422756130_full_lightcurve_bysectors.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\dani}[1]{\textcolor{blue}{(#1)}}$
$\newcommand{\review}[1]{\textcolor{red}{#1}}$
$\newcommand{\cmnt}[1]$
$\newcommand{\software}[1]{\texttt{\sc #1}}$
$\newcommand{\porb}{P_\mathrm{orb}}$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\tell}{\mathcal{T}}$
$\newcommand{\tlv}{^1}$
$\newcommand{\warwick}{^2}$
$\newcommand{\weissman}{^3}$
$\newcommand{\harvard}{^4}$
$\newcommand{\mpia}{^5}$
$\newcommand{\berkeley}{^3}$
$\newcommand{\samplesize}{15 779}$
$\newcommand{\inputsize}{4 301 148}$
$\newcommand{\longinputsize}{8 975 643}$
$\newcommand{\scorethreshold}{0.6}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\dani$}[1]{\textcolor{blue}{(#1)}}$
$\newcommand{$\review$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\cmnt$}[1]$
$\newcommand{$\software$}[1]{\texttt{\sc #1}}$
$\newcommand{$\porb$}{P_\mathrm{orb}}$
$\newcommand{$\teff$}{T_\mathrm{eff}}$
$\newcommand{$\tell$}{\mathcal{T}}$
$\newcommand{$\tlv$}{^1}$
$\newcommand{$\warwick$}{^2}$
$\newcommand{$\weissman$}{^3}$
$\newcommand{$\harvard$}{^4}$
$\newcommand{$\mpia$}{^5}$
$\newcommand{$\berkeley$}{^3}$
$\newcommand{$\samplesize$}{15 779}$
$\newcommand{$\inputsize$}{4 301 148}$
$\newcommand{$\longinputsize$}{8 975 643}$
$\newcommand{$\scorethreshold$}{0.6}$</div>



<div id="title">

# : Orbital Periods, Binary Fraction, and Tertiary Companions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.06194-b31b1b.svg)](https://arxiv.org/abs/2211.06194)<mark>Appeared on: 2022-11-14</mark> - _27 pages, 21 figures, submitted to MNRAS. The associated sample data will be released alongside the accepted paper_

</div>
<div id="authors">

<mark><mark>Matthew J. Green</mark></mark>, et al. -- incl., <mark><mark>Hans-Walter Rix</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present a homogeneously-selected sample of$\samplesize$candidate binary systems with main sequence primary stars and orbital periods shorter than 5 days.The targets were selected from\textit{TESS}full-frame image lightcurves on the basis of their tidally-induced ellipsoidal modulation.Spectroscopic follow-up suggests a sample purity of$83 \pm 13$per cent.Injection-recovery tests allow us to estimate our overall completeness as$28 \pm 3$per cent with$\porb$$< 3$days and to quantify our selection effects.We estimate that$39 \pm 4$per cent of our sample are contact binary systems, and we disentangle the period distributions of the contact and detached binaries.We derive the orbital period distribution of the main sequence binary population at short orbital periods, finding a distribution continuous with the log-normal distribution previously found for solar-type stars at longer periods, but with a significant steepening  at$\porb$$\lesssim 3$days, and a pile-up of contact binaries at$\porb$$\approx 0.4$days.Companions in the period range 1--5 days are found to be an order of magnitude more frequent around stars hotter than$\approx 6250 K$(the Kraft break) when compared to cooler stars, suggesting that magnetic braking plays an important role in shaping the temperature and period distributions.We detect resolved tertiary companions to$9.0 \pm 0.2$per cent of our binaries with a median separation of 3200 AU.The frequency of tertiary companions rises to$29 \pm 5$per cent among the systems with the shortest ellipsoidal periods.This large binary sample with quantified selection effects will be a powerful resource for future studies of detached and contact binary systems with$\porb$$<5$days.$\cmnt${word count: 242/250}

</div>

<div id="div_fig1">

<img src="tmp_2211.06194/./figures/target_properties.png" alt="Fig2" width="100%"/>

**Figure 2. -** Properties of 93 targets with score $> 0.6$ that were observed with LCO spectroscopic follow-up. Nine targets with $\software${beer} scores $< 0.6$ have not been plotted.
The upper panel plots $\software${beer} score, the middle panel shows primary $T_\mathrm{eff}$ from TIC, and lower panel shows the second-order ellipsoidal amplitude, all as a function of orbital period.
Coloured points indicate targets observed with LCOGT, coloured according to whether or not they showed RV variability, while grey points show the other targets in our sample (before applying the cut in Equation \ref{eq:lco-cut}).
The cut in Equation \ref{eq:lco-cut} is shown as a dashed line in the lower panel, which separates the lower-left region populated primarily by targets that are not RV variable.
 (*fig:lco-targets*)

</div>
<div id="div_fig2">

<img src="tmp_2211.06194/./figures/period_distribution_comparison.png" alt="Fig12" width="100%"/>

**Figure 12. -** A comparison of the orbital period distribution of our sample with several eclipsing binary samples, as summarized in Table \ref{tab:ebs-samples}.
Between one and three days our period distribution is similar to those of other samples.
 (*fig:ebs_comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2211.06194/./figures/contact_fraction_period.png" alt="Fig6" width="100%"/>

**Figure 6. -** \textit{Top:} The fraction of lightcurves with $R_{23}$ greater than the cut in \ref{eq:contacts} as a function of orbital period, for our real sample and for simulated lightcurves of detached and contact binary systems.
\textit{Below:} The fraction of the real sample which must be contact binary systems as a function of orbital period, in order to reproduce the curve of the real sample in the top figure.
The dotted grey line shows an approximation to the contact binary fraction described in Equation \ref{eq:contact-fraction}.
 (*fig:contact-fraction*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\}{MJ}$
$\newcommand{\}{ME}$
$\newcommand{\}{RJ}$
$\newcommand{\}{RE}$
$\newcommand{\}{RS}$
$\newcommand{\}{msol}$
$\newcommand{\}{kms}$
$\newcommand{\}{ms}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{\}{MJ}$
$\newcommand{\}{ME}$
$\newcommand{\}{RJ}$
$\newcommand{\}{RE}$
$\newcommand{\}{RS}$
$\newcommand{\}{msol}$
$\newcommand{\}{kms}$
$\newcommand{\}{ms}$</div>



<div id="title">

# A sub-Neptune planet around TOI-1695 discovered and characterized with SPIRou and TESS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.06205-b31b1b.svg)](https://arxiv.org/abs/2211.06205)<mark>Appeared on: 2022-11-14</mark> - _15 pages (+9 appendix pages), 19 figures, 8 tables_

</div>
<div id="authors">

F. Kiefer, et al. -- incl., <mark><mark>K. Hesse</mark></mark>, <mark><mark>H. Lewis</mark></mark>, <mark><mark>C.-F. Liu</mark></mark>

</div>
<div id="abstract">

**Abstract:** TOI-1695 is a V-mag=13 M-dwarf star from the northern hemisphere at 45 pc from the Sun, around which a 3.134-day periodic transit signal from a super-Earth candidate was identified in TESS photometry. With a transit depth of 1.3 mmag, the radius of candidate TOI-1695.01 was estimated by the TESS pipeline to be 1.82 R$_\oplus$with an equilibrium temperature of$\sim$620 K. We successfully detect a reflex motion of the star and establish it is due to a planetary companion at an orbital period consistent with the photometric transit period thanks to a year-long radial-velocity monitoring of TOI-1695 by the SPIRou infrared spectropolarimeter. We use and compare different methods to reduce and analyse those data. We report a 5.5-$\sigma$detection of the planetary signal, giving a mass of$5.5\pm1.0$M$_\oplus$and a radius of 2.03$\pm$0.18 R$_\oplus$. We derive a mean equilibrium planet temperature of 590$\pm$90 K. The mean density of this small planet of 3.6$\pm$1.1 g cm$^{-3}$is similar (1.7--$\sigma$lower) than that of the Earth. It leads to a non-negligible fraction of volatiles in its atmosphere with$f_{H,He}$=0.28$^{+0.46}_{-0.23}$\%or$f_\text{water}$=23$\pm$12\%. TOI-1695 b is a new sub-Neptune planet at the border of the M-dwarf radius valley that can help test formation scenarios for super-Earth/sub-Neptune-like planets.

</div>

<div id="div_fig1">

<img src="tmp_2211.06205/./toi_1695_gaia.png" alt="Fig4" width="100%"/>

**Figure 4. -** Spectral energy distribution of TOI-1695. Red symbols represent the observed photometric measurements, where the horizontal bars represent the effective width of the passband. Blue symbols are the model fluxes from the best-fit NextGen atmosphere model (black). The green curve is the published Gaia DR3 spectrum for this star. \label{fig:SED} (*fig:SED*)

</div>
<div id="div_fig2">

<img src="tmp_2211.06205/./TOI-1695_rvsvswin_DC_v6_periodogram.png" alt="Fig5.1" width="33%"/><img src="tmp_2211.06205/./TOI-1695_fwhmvswin_DC_v6_periodogram.png" alt="Fig5.2" width="33%"/><img src="tmp_2211.06205/./TOI-1695_bisvswin_DC_v6_periodogram.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** From top to bottom, RV, FWHM and BIS Lomb-Scargle periodograms with FAP levels 0.01 and 0.05 indicated as dotted and dashed lines for data extracted with the APERO v6 version. They are compared to the window function (red solid line). The vertical orange solid line indicates the 3.134-days period of the TESS transit signal. The moon synodic orbital period (29.53 days) is also shown as a green solid lines. \label{fig:periodo_all} (*fig:periodo_all*)

</div>
<div id="div_fig3">

<img src="tmp_2211.06205/./TIC422756130_full_lightcurve_bysectors.png" alt="Fig1" width="100%"/>

**Figure 1. -** The full TESS lightcurve separated in sectors (gray dots) and binned with a 0.1-day timestep (black). The identified transit locations are shown with orange vertical lines. \label{fig:transit_detection} (*fig:transit_detection*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

87  publications in the last 7 days.
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers